# Evaluating Arithmetic Operations Using Our Tiny Language Model

This notebook demonstrates how to use our tiny language model to evaluate simple arithmetic operations in Python code snippets. The process involves loading the model, preparing the necessary metadata for character-integer mappings, and defining functions for encoding and decoding strings. We then evaluate various code examples to observe the model's generated results. This workflow highlights the capabilities of our model in executing arithmetic expressions.


## Import Necessary Libraries

Import essential libraries for model loading, evaluation, and tokenization.

In [1]:
import os
import torch
import numpy as np
import pickle
import re
from model import GPT

## Load the Model

Set the model name and path, and load our tiny language model into the appropriate device (CPU or GPU).

In [2]:
model_name = "arithmetics_level1_696K"
model_path = os.path.join('models', f"{model_name}.pth")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GPT()
model = torch.compile(model)  
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

OptimizedModule(
  (_orig_mod): GPT(
    (token_embedding_table): Embedding(33, 96)
    (position_embedding_table): Embedding(256, 96)
    (blocks): Sequential(
      (0): Block(
        (sa): MultiHeadAttention(
          (heads): ModuleList(
            (0-5): 6 x Head(
              (key): Linear(in_features=96, out_features=16, bias=False)
              (query): Linear(in_features=96, out_features=16, bias=False)
              (value): Linear(in_features=96, out_features=16, bias=False)
              (dropout): Dropout(p=0, inplace=False)
            )
          )
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (ffwd): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=96, out_features=384, bias=False)
            (1): GELU(approximate='none')
            (2): Linear(in_features=384, out_features=96, bias=False)
            (3): Dropout(p=0, inplace=False)
          )


## Load Tokenizers

Load the metadata file containing the mappings from characters to integers and vice versa.

In [4]:
meta_path = os.path.join('data', 'meta.pkl')  
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)
# Récuperer le mapping Caractères-Entiers
stoi = meta['stoi']
itos = meta['itos']

# Fonctions pour la tokenisation
def encode(s):
    """
    Encode string `s` into token IDs.
    """
    return [stoi[c] for c in s]

def decode(l):
    """
    Decode token IDs `l` into a string.
"""
    return ''.join([itos[i] for i in l])
    

## Define Function to Evaluate Code Example

Implement a function to evaluate a given code snippet using our model, and extract the result from the model's output.

In [5]:
# Fonction pour donner au modèle un extrait de code à executer 
def evaluate_example(example , max_new_tokens=22):
    
    encoded_example = torch.tensor(encode(example), dtype=torch.long).unsqueeze(0).to(device)
    
    with torch.no_grad():
        response = decode(model.generate(encoded_example, max_new_tokens=max_new_tokens)[0].tolist())
    splited_response = response.split("# output")
    result_response = splited_response[-1]
    generated_results = [float(match.group()) for match in re.finditer(r"(?<=# )-?\d+(\.\d+)?", result_response.split('\n\n')[0].replace("\n", ""))]

    return  generated_results


## Examples

In [6]:
# Exemple 1
eval_prompt_1 = """
a = 5
b = 7 
c = a * b
print(c)
# output
"""

print("Example 1:")
print(eval_prompt_1)
generated_results_1 = evaluate_example(eval_prompt_1)
print("Generated Result:", generated_results_1[0])


Example 1:

a = 5
b = 7 
c = a * b
print(c)
# output

Generated Result: 35.0


In [9]:
# Exemple 2
eval_prompt_2 = """
e = 10
c = 10
a = e / c
print(a)
# output
"""

print("Example 2:")
print(eval_prompt_2)
generated_results_2 = evaluate_example(eval_prompt_2)
print("Generated Result:", generated_results_2[0])


Example 2:

e = 10
c = 10
a = e / c
print(a)
# output

Generated Result: 1.0


In [10]:
# Exemple 3
eval_prompt_3 = """
a = 9
d = 3
d = 1
d = 5 + 8
print(d * 5)
# output
"""

print("Example 3:")
print(eval_prompt_3)
generated_results_3 = evaluate_example(eval_prompt_3)
print("Generated Result:", generated_results_3[0])

Example 3:

a = 9
d = 3
d = 1
d = 5 + 8
print(d * 5)
# output

Generated Result: 90.0
